## Stats507 Homework 3, Winter 2020
### Diana Liang
#### dliangia@umich.edu

1. Problem 1 took me 1 hour.
2. Problem 2 took me 1 hour.
3. Problem 3 took me 1.5 hours.
4. Problem 4 took me 0.75 hours.
5. Problem 5 took me 0.75 hours.
6. Problem 6 took me 0.5 hours.

In [1]:
# you might need to install pycodestyle_magic and flake8
%load_ext pycodestyle_magic
%pycodestyle_on

### Problem 1: Counting Word Bigrams
1. Write a function count_bigrams_in_file that takes a filename as its only argument.

In [2]:
import string


def count_bigrams_in_file(file_name):
    # check if file is proper for function
    try:
        f = open(file_name)
    except:
        if type(file_name) is str:
            raise ValueError("File cannot be opened")
        else:
            raise TypeError("Input not a file name as string")
    # create a list of all the words
    word_list = []
    for line in f:
        for word in line.split():
            word = word.lower()
            word = word.translate(str.maketrans('', '', string.punctuation))
            word_list.append(word)
    # create dictionary of word bigrams
    d = dict()
    for i in range(len(word_list)-1):
        temp = (word_list[i], word_list[i+1])
        if temp in d:
            d[temp] += 1
        else:
            d[temp] = 1
    f.close()
    return d

8:5: E722 do not use bare 'except'


In [3]:
with open('./test.txt', 'w') as f:
    f.write('Half a league, half a league,\n Half a league onward.')

count_bigrams_in_file('test.txt') == {('a', 'league'): 3,
                                      ('half', 'a'): 3,
                                      ('league', 'half'): 2,
                                      ('league', 'onward'): 1}

True

2. Run your function on this file, and pickle the resulting dictionary in a file called mb.bigrams.pickle.

In [4]:
import pickle


obj = count_bigrams_in_file('WandP.txt')
new_file = open('mb.bigrams.pickle', 'wb')
pickle.dump(obj, new_file)
new_file.close()

3. Write a function collocations that takes a file name as its only argument and returns a dictionary.

In [5]:
def collocations(file_name):
    try:
        f = open(file_name)
    except:
        if type(file_name) is str:
            raise ValueError("File cannot be opened")
        else:
            raise TypeError("Input not a file name as string")
    word_list = []
    for line in f:
        for word in line.split():
            word = word.lower()
            word = word.translate(str.maketrans('', '', string.punctuation))
            word_list.append(word)
    bigram = []
    for i in range(len(word_list)-1):
        temp = (word_list[i], word_list[i+1])
        temp2 = (word_list[i+1], word_list[i])
        bigram.append(temp)
        bigram.append(temp2)
    bigram = set(bigram)
    d = dict()
    for bg in bigram:
        if bg[0] not in d:
            d[bg[0]] = []
        if bg[1] not in d[bg[0]]:
            d[bg[0]].append(bg[1])
    for each in d:
        d[each] = set(d[each])
    return d

4:5: E722 do not use bare 'except'


In [6]:
collocations('test.txt') == {'half': {'league', 'a'},
                             'a': {'half', 'league'},
                             'league': {'onward', 'half', 'a'},
                             'onward': {'league'}}

True

4. Run your function on the file WandP.txt and pickle the resulting dictionary in a file called mb.colloc.pickle.

In [7]:
obj = collocations('WandP.txt')
new_file = open('mb.colloc.pickle', 'wb')
pickle.dump(obj, new_file)
new_file.close()

### Problem 2: More Fun with Vectors
#### Define the class Vector in one code cell below, since class can't be declared several times.

In [23]:
class Vector:
    '''Represents a vector'''

    def __init__(self, dim=0, vt=()):
        if type(dim) is not float and type(dim) is not int:
            raise TypeError("dim is not numeric")
        if len(vt) != dim:
            raise ValueError("dim does not fit length of Vector")
        self.dim = dim
        self.vt = vt

    def __eq__(self, other):
        if self.dim != other.dim:
            raise ValueError("Vectors have different dimensions")
        return(self.vt == other.vt)

    def __lt__(self, other):
        if self.dim != other.dim:
            raise ValueError("Vectors have different dimensions")
        return(self.vt < other.vt)

    def __gt__(self, other):
        if self.dim != other.dim:
            raise ValueError("Vectors have different dimensions")
        return(self.vt > other.vt)

    def __le__(self, other):
        if self.dim != other.dim:
            raise ValueError("Vectors have different dimensions")
        return(self.vt <= other.vt)

    def __ge__(self, other):
        if self.dim != other.dim:
            raise ValueError("Vectors have different dimensions")
        return(self.vt >= other.vt)

    def dot(self, other):
        if isinstance(other, Vector):
            pass
        else:
            raise TypeError("Input is not a Vector")
        if self.dim != other.dim:
            raise ValueError("Vectors have different dimensions")
        final = 0
        for i in range(self.dim):
            temp = self.vt[i] * other.vt[i]
            final = final + temp
        return final

    def __mul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            final = tuple([x * other for x in self.vt])
            return(Vector(self.dim, final))
        elif isinstance(other, Vector):
            if self.dim != other.dim:
                raise ValueError("Vectors have different dimensions")
            else:
                final = []
                for i in range(self.dim):
                    temp = self.vt[i] * other.vt[i]
                    final.append(temp)
                return(Vector(self.dim, tuple(final)))
        else:
            raise TypeError("Input cannot be multiplied with Vector")

    def __rmul__(self, other):
        if isinstance(other, float) or isinstance(other, int):
            final = tuple([x * other for x in self.vt])
            return(Vector(self.dim, final))
        elif isinstance(other, Vector):
            if self.dim != other.dim:
                raise ValueError("Vectors have different dimensions")
            else:
                final = []
                for i in range(self.dim):
                    temp = self.vt[i] * other.vt[i]
                    final.append(temp)
                return(Vector(self.dim, tuple(final)))
        else:
            raise TypeError("Input cannot be multiplied with Vector")

    def norm(self, p):
        if isinstance(p, int) and isinstance(p, float):
            raise TypeError("Input is not numeric")
        if p < 0:
            raise ValueError("Input is not non-negative")
        x = tuple(map(abs, self.vt))
        if p == 0:
            ones = 0
            for i in self.vt:
                if i != 0:
                    ones += 1
            return(float(ones))
        elif p == float('inf'):
            big = max(self.vt)
            return(float(big))
        else:
            temp = tuple(map(lambda x: x**p, self.vt))
            temp = sum(temp)**(1/p)
            return(temp)

2.2: The vector entries are a tuple since the exact position of each element matters in a vector.

2.3: The dimension and entries are instance attributes because they should be distinct to that specifically created vector rather to all the vectors created under this class.

In [15]:
# 2.1
print(isinstance(Vector(3, (1, 2, 3)), Vector))

# 2.4
print((Vector(3, (1, 2, 3)) > Vector(3, (1, 2, 2))) == (True))
print((Vector(3, (1, 2, 3)) >= Vector(3, (1, 2, 2))) == (True))
print((Vector(3, (1, 2, 3)) < Vector(3, (1, 2, 2))) == (False))
print((Vector(3, (1, 2, 3)) <= Vector(3, (1, 2, 2))) == (False))
print((Vector(3, (1, 2, 3)) == Vector(3, (1, 2, 2))) == (False))

# 2.5
print((Vector(3, (1, 2, 3)).dot(Vector(3, (1, 2, 2)))) == 11.0)

# 2.6
print(2 * Vector(3, (1, 2, 3)) == Vector(3, (2, 4, 6)))
print(Vector(3, (1, 2, 3)) * 2 == Vector(3, (2, 4, 6)))

# 2.7
print(Vector(3, (1, 2, 3)).norm(1) == 6.0)

True
True
True
True
True
True
True
True
True
True


###  Problem 3: Iterators and Generators
1. Define a class Fibo of iterators that enumerate the Fibonacci numbers.

In [17]:
class Fibo:
    def __init__(self):
        self.fo = 0
        self.f1 = 1

    def __next__(self):
        out = self.fo
        (self.fo, self.f1) = (self.f1, self.fo + self.f1)
        return(out)

    def __iter__(self):
        return(self)

In [18]:
ans = Fibo()
[next(ans) for _ in range(10)] == [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

True

2. Define a class GenFibo of iterators that enumerate generalized Fibonacci numbers.

In [22]:
class GenFibo:
    def __init__(self, fo=0, f1=1):
        self.fo = fo
        self.f1 = f1

    def __next__(self):
        out = self.fo
        (self.fo, self.f1) = (self.f1, self.fo + self.f1)
        return(out)

    def __iter__(self):
        return(self)

In [24]:
ans = GenFibo(2, 1)
[next(ans) for _ in range(10)] == [2, 1, 3, 4, 7, 11, 18, 29, 47, 76]

True

3. Define a generator primes that enumerates the prime numbers.
#### You may use the function is_prime that we defined in class (or something similar to it), but such solutions will not receive full credit

In [26]:
def primes():
    num = int(2)
    num_list = []
    while True:
        div_list = list(map(lambda x: num % x, num_list))
        if all(div_list) != 0:
            yield num
        num_list.append(num)
        num += 1

In [27]:
ans = primes()
[next(ans) for _ in range(10)] == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]

True

4. Define a generator ulam that enumerates the Ulam numbers.

In [28]:
def ulam():
    ulam_list = []
    sum_list = []
    u = 1
    while True:
        if sum(list(map(lambda x: x == u, sum_list))) < 2:
            for i in range(len(ulam_list)):
                sum_list.append(ulam_list[i] + u)
            ulam_list.append(u)
            yield u
        u += 1

In [29]:
ans = ulam()
[next(ans) for _ in range(10)] == [1, 2, 3, 4, 6, 8, 11, 13, 16, 18]

True

###  Problem 4: List Comprehensions and Generator Expressions (5 points)
1. Write a list comprehension that enumerates the sequence $3^n - 1$ for n = 1, 2, 3, ..., 20.

In [31]:
pow3minus1 = [(3**n)-1 for n in range(1, 21)]

2. Write a generator expression that enumerates the Lazy Caterer's sequence. Please assign this generator expression to a variable called caterer.

In [32]:
def nonneg_int():
    n = 0
    while True:
        yield n
        n += 1

In [33]:
caterer = (((n**2)+n+2)/2 for n in nonneg_int())

3. Write a generator expression that enumerates the pyramid numbers.

In [34]:
def pos_int():
    n = 1
    while True:
        yield n
        n += 1

In [35]:
pyramid = ((n*(n+1)*(2*n+1))/6 for n in pos_int())

4. Write a generator expression that enumerates the octahedral numbers.

In [36]:
octe = ((n*(2*n**2+1))/3 for n in pos_int())

### Problem 5: Map, Filter and Reduce
1. Write a one-line expression that computes the sum of the first 10 odd  (starting from 1).

In [37]:
sum_of_odd_numbers = sum(filter(lambda x: x % 2 == 1, range(1, 20)))

2. Write a one-line expression that computes the product of the first 13 primes. You may use the primes generator that you defined above. Please assign the output of this expression to a variable called product_of_primes.

In [40]:
import functools

n = primes()
product_of_primes = functools.reduce(lambda x, y: x*y, [next(n) for _ in range(13)])

4:80: E501 line too long (84 > 79 characters)


3. Write a one-line expression that computes the sum of the squares of the first 31 primes. You may use the primes generator that you defined above. Please assign the output of this expression to a variable called squared_primes.

In [41]:
n = primes()
squared_primes = sum(list(map(lambda x: x**2, [next(n) for _ in range(31)])))

4. Write a one-line expression that computes a list of the first twenty harmonic numbers. Please assign the output of this expression to a variable called harmonics.

In [43]:
harmonics = list(map(lambda n: sum([1/k for k in range(1, n+1)]), range(1, 21)))

1:80: E501 line too long (80 > 79 characters)


5. Write a one-line expression that computes the geometric mean of the first 12 octahedral numbers.

In [45]:
octa_geom = (functools.reduce(lambda x, y: x*y, [(n*(2*(n**2)+1))/3 for n in range(1, 4)]))**(1/3)

1:80: E501 line too long (98 > 79 characters)


### Problem 6: Fun with Polynomials
1. Write a function make_poly that takes a list of numbers (ints and/or foats) coeffs as its only argument and returns a function p.

In [47]:
def make_poly(coeffs):
    def p(x, coeffs=coeffs):
        poly_list = []
        for i in range(len(coeffs)):
            poly_list.append(coeffs[i]*(x**i))
        return(sum(poly_list))
    return p

In [48]:
make_poly([1, 3, 4])(2) == 23

True

2. Write a function eval_poly that takes two lists of numbers (ints and/or foats), coeffs and args.

In [49]:
def eval_poly(coeffs, args):
    result_list = []
    for i in range(len(args)):
        result_list.append(make_poly(coeffs)(args[i]))
    return result_list

In [50]:
eval_poly([1, 3, 4], [2, 3]) == [23, 46]

True